# MQ-3 

## Ro calculation

### Specifications

#### Standard Work Condition

    Vc (Circuit Voltage)       :  5v +/- 0.1
    Vh (Heating Voltage)       :  5v +/- 0.1
    Rl (Load Resistance)       :  200kOhm
    Rh (Heater Resistance)     :  33 ohm +/- 5%
    Ph (Heating consumption)   :  less than 750mw
    

#### Sensitivity Characteristic:

    Rs (Sensing Resistance)    : 1MOhm  -  8MOhm (0.4mg/L alcohol)
    Preheat Time               : Over 24 hour
    
    Detecting Concentration  Scope:
        Alcohol                :  0.05mg/L  - 10 mg/L


In [1]:
#%pip install numpy

import numpy as np
import math

In [2]:
## Gas concentration values referenced in the datasheet

alcohol = {
    'p1': {'x': 0.100,  'y': 2.332},
    'p2': {'x': 0.216,  'y': 1.539},
    'p3': {'x': 0.402,  'y': 1.025},
    'p4': {'x': 1.008,  'y': 0.542},
    'p5': {'x': 1.646,  'y': 0.396},
    'p6': {'x': 2.535,  'y': 0.289},
    'p7': {'x': 3.998,  'y': 0.206},
    'p8': {'x': 6.044,  'y': 0.163},
    'p9': {'x': 8.090,  'y': 0.131}, 
    'p10':{'x': 10.000, 'y': 0.114}, 
}

benzine = {
    'p1': {'x': 0.100,  'y': 4.133},
    'p2': {'x': 0.214,  'y': 3.192},
    'p3': {'x': 0.409,  'y': 2.581},
    'p4': {'x': 1.030,  'y': 1.767},
    'p5': {'x': 1.650,  'y': 1.469},
    'p6': {'x': 2.535,  'y': 1.222},
    'p7': {'x': 4.084,  'y': 1.025},
    'p8': {'x': 6.207,  'y': 0.917},
    'p9': {'x': 8.047,  'y': 0.821}, 
    'p10':{'x': 10.000, 'y': 0.784}, 
}

# Methane
ch4 = {
    'p1': {'x': 0.100,  'y': 49.566},
    'p2': {'x': 0.217,  'y': 48.659},
    'p3': {'x': 0.412,  'y': 46.464},
    'p4': {'x': 1.035,  'y': 44.778},
    'p5': {'x': 1.650,  'y': 42.365},
    'p6': {'x': 2.535,  'y': 42.758},
    'p7': {'x': 4.105,  'y': 41.590},
    'p8': {'x': 6.207,  'y': 40.453},
    'p9': {'x': 8.090,  'y': 40.453}, 
    'p10':{'x': 10.000, 'y': 39.713}, 
}

hexane = {
    'p1': {'x': 0.100,  'y': 51.432},
    'p2': {'x': 0.217,  'y': 39.713},
    'p3': {'x': 0.414,  'y': 30.949},
    'p4': {'x': 1.041,  'y': 22.609},
    'p5': {'x': 6.207,  'y': 12.520},
    'p6': {'x': 8.090,  'y': 11.001},
    'p7': {'x': 10.000, 'y': 10.031},
}


# Liquefied Petroleum Gas
lpg = {
    'p1': {'x': 0.100,  'y': 51.432},
    'p2': {'x': 0.216,  'y': 43.959},
    'p3': {'x': 0.414,  'y': 36.210},
    'p4': {'x': 1.041,  'y': 29.826},
    'p5': {'x': 6.240,  'y': 16.065},
    'p6': {'x': 8.047,  'y': 13.987},
    'p7': {'x': 10.000, 'y': 13.111},
}


# Carbon Monoxide
co = {
    'p1': {'x': 0.100,  'y': 51.909},
    'p2': {'x': 0.216,  'y': 45.613},
    'p3': {'x': 0.414,  'y': 41.975},
    'p4': {'x': 1.035,  'y': 33.943},
    'p5': {'x': 6.174,  'y': 20.237},
    'p6': {'x': 8.026,  'y': 18.198},
    'p7': {'x': 10.000, 'y': 16.214},
}

AIR = 60

In [3]:
# RS value calculation
Rs    = lambda Vcc, Vrl, RL : ((Vcc * RL) / Vrl) - RL

# R0 value calculation
R0    = lambda Rs, air : Rs / air

# RS/Ro value calculation (ratio)
Ratio = lambda Rs, R0: Rs / R0

# log(y) = m.log(x) + b
# m
M     = lambda p1, p2 : (math.log10( p2['y'] / p1['y'] )) / (math.log10(p2['x']/p1['x']))
# b
B     = lambda p, m : math.log10(p['y']) - (m * math.log10(p['x']))

# mg/L value calculation 
MgL   = lambda  y, m, b: 10 ** ((math.log10(y) - b) / m)

# PPM value calculation 
PPM   = lambda y, m, b: MgL(y, m, b) * 1000

In [4]:
## Calculate the points to be used for the calculation of _m_ and _b_ according to the _ratio_ value 
def getPoints(racio, values):
    size = len(values)
    for idx in range(size - 1):
        if racio >= values[idx]['y']:
            if idx == 0:
                return (values[0], values[1])
            return(values[idx - 1], values[idx])
    return(values[size - 2], values[size - 1])

In [5]:
points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10']

_alcohol = [ alcohol[n] for n in list(alcohol) if n in points]
_benzine = [ benzine[n] for n in list(benzine) if n in points]
_ch4     = [ ch4[n]     for n in list(ch4)     if n in points]
_hexane  = [ hexane[n]  for n in list(hexane)  if n in points]
_lpg     = [ lpg[n]     for n in list(lpg)     if n in points]
_co      = [ co[n]      for n in list(co)      if n in points]

In [6]:
_ratios      = [100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 
                9, 8, 7, 6, 5, 4, 3, 2, 1, 
                0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

alcohol_mgl = []
benzine_mgl = []
ch4_mgl     = []
hexane_mgl  = []
lpg_mgl     = []
co_mgl      = []


for r in _ratios:
    p1, p2  = getPoints(r, _alcohol)
    m = M(p1,p2)
    b = B(p1, m)
    alcohol_mgl.append(MgL(r, m, b))
    
    p1, p2  = getPoints(r, _benzine)
    m = M(p1,p2)
    b = B(p1, m)
    benzine_mgl.append(MgL(r, m, b))
    
    p1, p2  = getPoints(r, _ch4)
    m = M(p1,p2)
    b = B(p1, m)
    ch4_mgl.append(MgL(r, m, b))
    
    p1, p2  = getPoints(r, _hexane)
    m = M(p1,p2)
    b = B(p1, m)
    hexane_mgl.append(MgL(r, m, b))
    
    p1, p2  = getPoints(r, _lpg)
    m = M(p1,p2)
    b = B(p1, m)
    lpg_mgl.append(MgL(r, m, b))
    
    p1, p2  = getPoints(r, _co)
    m = M(p1,p2)
    b = B(p1, m)
    co_mgl.append(MgL(r, m, b))


    
print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Ratio','Alcohol', 'Benzine', 'CH4', 'Hexane', 'LPG', 'CO'))
for i in range(len(_ratios)):
    if _ratios[i] > 10:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(alcohol_mgl[i] * 1000 ,3),
                       round(benzine_mgl[i] * 1000,3),
                       round(ch4_mgl[i] * 1000,3),
                       round(hexane_mgl[i] * 1000,3),
                       round(lpg_mgl[i] * 1000,3),
                       round(co_mgl[i] * 1000,3)
                      ))
    else:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format( _ratios[i],
               round(alcohol_mgl[i] * 1000,3),
               round(benzine_mgl[i] * 1000,3),
               ' - - -',
               round(hexane_mgl[i] * 1000,3),
               round(lpg_mgl[i] * 1000,3),
               round(co_mgl[i] * 1000,3)
              ))



Ratio    Alcohol       Benzine       CH4           Hexane        LPG           CO           
100      0.094         0.008         0.0           13.641        3.833         2.014        
90       0.115         0.011         0.0           18.704        6.427         3.772        
80       0.143         0.016         0.0           26.619        11.453        7.606        
70       0.183         0.024         0.0           39.713        22.049        16.849       
60       0.243         0.038         0.033         63.025        46.964        42.2         
50       0.341         0.065         69.371        108.828       114.856       125.003      
40       0.516         0.125         9206.55       212.369       296.458       509.681      
30       0.88          0.292         250300.721    453.648       1012.607      1585.395     
20       1.864         0.963         26307904.547  1507.735      3309.757      6356.303     
10       6.736         7.413          - - -        10071.326     24843

### Empty $V_{RL}$ Values

The minimum $V_{RL}$ voltage values within the obtained values were considered. 
The measurement took place for +/- 4 days in an empty room with the door closed (no one entered the room) and the windows open to ensure air circulation.

- __Sensor 1__: 0.50 Volts
- __Sensor 2__: 0.48 Volts

In [7]:
### MQ 3 - sensor 1
sensor1 = {}
sensor1['Vcc'] = 5      #volts
sensor1['Vrl'] = 0.50   #volts
sensor1['RL']  = 10     #kOhms
sensor1['AIR'] = AIR   

sensor1['rs'] = Rs(sensor1['Vcc'], sensor1['Vrl'], sensor1['RL'])
sensor1['ro'] = R0(sensor1['rs'],  sensor1['AIR'])
sensor1['ratio'] = Ratio(sensor1['rs'], sensor1['ro'])

# MQ 3 - sensor 2
sensor2 = {}
sensor2['Vcc'] = 5      #volts
sensor2['Vrl'] = 0.48   #volts
sensor2['RL']  = 10     #kOhms
sensor2['AIR'] = AIR   

sensor2['rs'] = Rs(sensor2['Vcc'], sensor2['Vrl'], sensor2['RL'])
sensor2['ro'] = R0(sensor2['rs'],  sensor2['AIR'])
sensor2['ratio'] = Ratio(sensor2['rs'], sensor2['ro'])

print ("{:<8} {:<8} {:<8} {:<8}"
       .format('Sensor','Rs', 'Ro','Ratio'))

print ("{:<8} {:<8} {:<8} {:<8}"
       .format(1, 
               round(sensor1['rs'],3), 
               round(sensor1['ro'], 3), 
               round(sensor1['ratio'], 3)))

print ("{:<8} {:<8} {:<8} {:<8}"
       .format(2, 
               round(sensor2['rs'],3), 
               round(sensor2['ro'], 3), 
               round(sensor2['ratio'], 3)))

Sensor   Rs       Ro       Ratio   
1        90.0     1.5      60.0    
2        94.167   1.569    60.0    


In [8]:
## mg/L
# Concentration of gases in mg/L for the values of Rs/Ro (ratio) in Clean Air

sensor1['alcohol'] = {}
sensor1['alcohol']['p1'], sensor1['alcohol']['p2'] = getPoints(sensor1['ratio'], _alcohol)
sensor1['alcohol']['m'] = M(sensor1['alcohol']['p1'], sensor1['alcohol']['p2'])
sensor1['alcohol']['b'] = B(sensor1['alcohol']['p1'], sensor1['alcohol']['m'])


sensor1['benzine'] = {}
sensor1['benzine']['p1'], sensor1['benzine']['p2'] = getPoints(sensor1['ratio'], _benzine)
sensor1['benzine']['m'] = M(sensor1['benzine']['p1'], sensor1['benzine']['p2'])
sensor1['benzine']['b'] = B(sensor1['benzine']['p1'], sensor1['benzine']['m'])


sensor1['ch4'] = {}
sensor1['ch4']['p1'], sensor1['ch4']['p2'] = getPoints(sensor1['ratio'], _ch4)
sensor1['ch4']['m'] = M(sensor1['ch4']['p1'], sensor1['ch4']['p2'])
sensor1['ch4']['b'] = B(sensor1['ch4']['p1'], sensor1['ch4']['m'])


sensor1['hexane'] = {}
sensor1['hexane']['p1'], sensor1['hexane']['p2'] = getPoints(sensor1['ratio'], _hexane)
sensor1['hexane']['m'] = M(sensor1['hexane']['p1'], sensor1['hexane']['p2'])
sensor1['hexane']['b'] = B(sensor1['hexane']['p1'], sensor1['hexane']['m'])


sensor1['lpg'] = {}
sensor1['lpg']['p1'], sensor1['lpg']['p2'] = getPoints(sensor1['ratio'], _lpg)
sensor1['lpg']['m'] = M(sensor1['lpg']['p1'], sensor1['lpg']['p2'])
sensor1['lpg']['b'] = B(sensor1['lpg']['p1'], sensor1['lpg']['m'])


sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])

####

sensor2['alcohol'] = {}
sensor2['alcohol']['p1'], sensor2['alcohol']['p2'] = getPoints(sensor2['ratio'], _alcohol)
sensor2['alcohol']['m'] = M(sensor2['alcohol']['p1'], sensor2['alcohol']['p2'])
sensor2['alcohol']['b'] = B(sensor2['alcohol']['p1'], sensor2['alcohol']['m'])


sensor2['benzine'] = {}
sensor2['benzine']['p1'], sensor2['benzine']['p2'] = getPoints(sensor2['ratio'], _benzine)
sensor2['benzine']['m'] = M(sensor2['benzine']['p1'], sensor2['benzine']['p2'])
sensor2['benzine']['b'] = B(sensor2['benzine']['p1'], sensor2['benzine']['m'])


sensor2['ch4'] = {}
sensor2['ch4']['p1'], sensor2['ch4']['p2'] = getPoints(sensor2['ratio'], _ch4)
sensor2['ch4']['m'] = M(sensor2['ch4']['p1'], sensor2['ch4']['p2'])
sensor2['ch4']['b'] = B(sensor2['ch4']['p1'], sensor2['ch4']['m'])


sensor2['hexane'] = {}
sensor2['hexane']['p1'], sensor2['hexane']['p2'] = getPoints(sensor2['ratio'], _hexane)
sensor2['hexane']['m'] = M(sensor2['hexane']['p1'], sensor2['hexane']['p2'])
sensor2['hexane']['b'] = B(sensor2['hexane']['p1'], sensor2['hexane']['m'])


sensor2['lpg'] = {}
sensor2['lpg']['p1'], sensor2['lpg']['p2'] = getPoints(sensor2['ratio'], _lpg)
sensor2['lpg']['m'] = M(sensor2['lpg']['p1'], sensor2['lpg']['p2'])
sensor2['lpg']['b'] = B(sensor2['lpg']['p1'], sensor2['lpg']['m'])


sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])

####

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio','Alcohol', 'Benzine', 'CH4', 'Hexane', 'LPG', 'CO'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(MgL(sensor1['ratio'], sensor1['alcohol']['m'], sensor1['alcohol']['b']),3),
           round(MgL(sensor1['ratio'], sensor1['benzine']['m'], sensor1['benzine']['b']),3),
           round(MgL(sensor1['ratio'], sensor1['ch4']['m'],     sensor1['ch4']['b']),3),
           round(MgL(sensor1['ratio'], sensor1['hexane']['m'],  sensor1['hexane']['b']),3),
           round(MgL(sensor1['ratio'], sensor1['lpg']['m'],     sensor1['lpg']['b']),3),
           round(MgL(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),3),
          ))


print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(MgL(sensor2['ratio'], sensor2['alcohol']['m'], sensor2['alcohol']['b']),3),
           round(MgL(sensor2['ratio'], sensor2['benzine']['m'], sensor2['benzine']['b']),3),
           round(MgL(sensor2['ratio'], sensor2['ch4']['m'],     sensor2['ch4']['b']),3),
           round(MgL(sensor2['ratio'], sensor2['hexane']['m'],  sensor2['hexane']['b']),3),
           round(MgL(sensor2['ratio'], sensor2['lpg']['m'],     sensor2['lpg']['b']),3),
           round(MgL(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),3),
          ))


Volt     Ratio    Alcohol       Benzine       CH4           Hexane        LPG           CO           
0.5      60.0     0.0           0.0           0.0           0.063         0.047         0.042        
0.48     60.0     0.0           0.0           0.0           0.063         0.047         0.042        


In [9]:
## PPM
# Concentration of gases in PPM for the values of Rs/Ro (ratio) in Clean Air

sensor1['alcohol'] = {}
sensor1['alcohol']['p1'], sensor1['alcohol']['p2'] = getPoints(sensor1['ratio'], _alcohol)
sensor1['alcohol']['m'] = M(sensor1['alcohol']['p1'], sensor1['alcohol']['p2'])
sensor1['alcohol']['b'] = B(sensor1['alcohol']['p1'], sensor1['alcohol']['m'])


sensor1['benzine'] = {}
sensor1['benzine']['p1'], sensor1['benzine']['p2'] = getPoints(sensor1['ratio'], _benzine)
sensor1['benzine']['m'] = M(sensor1['benzine']['p1'], sensor1['benzine']['p2'])
sensor1['benzine']['b'] = B(sensor1['benzine']['p1'], sensor1['benzine']['m'])


sensor1['ch4'] = {}
sensor1['ch4']['p1'], sensor1['ch4']['p2'] = getPoints(sensor1['ratio'], _ch4)
sensor1['ch4']['m'] = M(sensor1['ch4']['p1'], sensor1['ch4']['p2'])
sensor1['ch4']['b'] = B(sensor1['ch4']['p1'], sensor1['ch4']['m'])


sensor1['hexane'] = {}
sensor1['hexane']['p1'], sensor1['hexane']['p2'] = getPoints(sensor1['ratio'], _hexane)
sensor1['hexane']['m'] = M(sensor1['hexane']['p1'], sensor1['hexane']['p2'])
sensor1['hexane']['b'] = B(sensor1['hexane']['p1'], sensor1['hexane']['m'])


sensor1['lpg'] = {}
sensor1['lpg']['p1'], sensor1['lpg']['p2'] = getPoints(sensor1['ratio'], _lpg)
sensor1['lpg']['m'] = M(sensor1['lpg']['p1'], sensor1['lpg']['p2'])
sensor1['lpg']['b'] = B(sensor1['lpg']['p1'], sensor1['lpg']['m'])


sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])

####

sensor2['alcohol'] = {}
sensor2['alcohol']['p1'], sensor2['alcohol']['p2'] = getPoints(sensor2['ratio'], _alcohol)
sensor2['alcohol']['m'] = M(sensor2['alcohol']['p1'], sensor2['alcohol']['p2'])
sensor2['alcohol']['b'] = B(sensor2['alcohol']['p1'], sensor2['alcohol']['m'])


sensor2['benzine'] = {}
sensor2['benzine']['p1'], sensor2['benzine']['p2'] = getPoints(sensor2['ratio'], _benzine)
sensor2['benzine']['m'] = M(sensor2['benzine']['p1'], sensor2['benzine']['p2'])
sensor2['benzine']['b'] = B(sensor2['benzine']['p1'], sensor2['benzine']['m'])


sensor2['ch4'] = {}
sensor2['ch4']['p1'], sensor2['ch4']['p2'] = getPoints(sensor2['ratio'], _ch4)
sensor2['ch4']['m'] = M(sensor2['ch4']['p1'], sensor2['ch4']['p2'])
sensor2['ch4']['b'] = B(sensor2['ch4']['p1'], sensor2['ch4']['m'])


sensor2['hexane'] = {}
sensor2['hexane']['p1'], sensor2['hexane']['p2'] = getPoints(sensor2['ratio'], _hexane)
sensor2['hexane']['m'] = M(sensor2['hexane']['p1'], sensor2['hexane']['p2'])
sensor2['hexane']['b'] = B(sensor2['hexane']['p1'], sensor2['hexane']['m'])


sensor2['lpg'] = {}
sensor2['lpg']['p1'], sensor2['lpg']['p2'] = getPoints(sensor2['ratio'], _lpg)
sensor2['lpg']['m'] = M(sensor2['lpg']['p1'], sensor2['lpg']['p2'])
sensor2['lpg']['b'] = B(sensor2['lpg']['p1'], sensor2['lpg']['m'])


sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])

####

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio','Alcohol', 'Benzine', 'CH4', 'Hexane', 'LPG', 'CO'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(PPM(sensor1['ratio'], sensor1['alcohol']['m'], sensor1['alcohol']['b']),3),
           round(PPM(sensor1['ratio'], sensor1['benzine']['m'], sensor1['benzine']['b']),3),
           round(PPM(sensor1['ratio'], sensor1['ch4']['m'],     sensor1['ch4']['b']),3),
           round(PPM(sensor1['ratio'], sensor1['hexane']['m'],  sensor1['hexane']['b']),3),
           round(PPM(sensor1['ratio'], sensor1['lpg']['m'],     sensor1['lpg']['b']),3),
           round(PPM(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),3),
          ))


print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(PPM(sensor2['ratio'], sensor2['alcohol']['m'], sensor2['alcohol']['b']),3),
           round(PPM(sensor2['ratio'], sensor2['benzine']['m'], sensor2['benzine']['b']),3),
           round(PPM(sensor2['ratio'], sensor2['ch4']['m'],     sensor2['ch4']['b']),3),
           round(PPM(sensor2['ratio'], sensor2['hexane']['m'],  sensor2['hexane']['b']),3),
           round(PPM(sensor2['ratio'], sensor2['lpg']['m'],     sensor2['lpg']['b']),3),
           round(PPM(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),3),
          ))


Volt     Ratio    Alcohol       Benzine       CH4           Hexane        LPG           CO           
0.5      60.0     0.243         0.038         0.033         63.025        46.964        42.2         
0.48     60.0     0.243         0.038         0.033         63.025        46.964        42.2         


In [10]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
Vrl    = np.arange(sensor1['Vrl'], 5, 0.1)
RL     = 10 #kOhms

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in Vrl]
projection['ro'] = sensor1['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('','Volt', 'Ratio','Alcohol', 'Benzine', 'CH4', 'Hexane', 'LPG', 'CO'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])


    projection['benzine'] = {}
    projection['benzine']['p1'], projection['benzine']['p2'] = getPoints(r, _benzine)
    projection['benzine']['m'] = M(projection['benzine']['p1'], projection['benzine']['p2'])
    projection['benzine']['b'] = B(projection['benzine']['p1'], projection['benzine']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['hexane'] = {}
    projection['hexane']['p1'], projection['hexane']['p2'] = getPoints(r, _hexane)
    projection['hexane']['m'] = M(projection['hexane']['p1'], projection['hexane']['p2'])
    projection['hexane']['b'] = B(projection['hexane']['p1'], projection['hexane']['m'])


    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    if r > 9:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(Vrl[idx], 3),
                   round(r, 2),
                   round(MgL(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                   round(MgL(r, projection['benzine']['m'], projection['benzine']['b']),2),
                   round(MgL(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(MgL(r, projection['hexane']['m'], projection['hexane']['b']),2),
                   round(MgL(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(MgL(r, projection['co']['m'], projection['co']['b']),2),
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(Vrl[idx], 3),
                   round(r, 2),
                   round(MgL(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                   round(MgL(r, projection['benzine']['m'], projection['benzine']['b']),2),
                   "----",
                   round(MgL(r, projection['hexane']['m'], projection['hexane']['b']),2),
                   round(MgL(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(MgL(r, projection['co']['m'], projection['co']['b']),2),
                  ))    
    
    

    Volt     Ratio    Alcohol       Benzine       CH4           Hexane        LPG           CO           
0   0.5      60.0     0.0           0.0           0.0           0.06          0.05          0.04         
1   0.6      48.89    0.0           0.0           0.18          0.12          0.13          0.14         
2   0.7      40.95    0.0           0.0           5.17          0.2           0.27          0.46         
3   0.8      35.0     0.0           0.0           42.65         0.3           0.49          0.91         
4   0.9      30.37    0.0           0.0           217.41        0.44          0.96          1.52         
5   1.0      26.67    0.0           0.0           967.66        0.64          1.44          2.38         
6   1.1      23.64    0.0           0.0           3865.14       0.91          2.04          3.61         
7   1.2      21.11    0.0           0.0           14141.99      1.28          2.83          5.34         
8   1.3      18.97    0.0           0.0       

In [11]:
## Projection - Increasing VRL

# Sensor 2

Vcc    = 5 #volts
Vrl    = np.arange(sensor2['Vrl'],5, 0.1)
RL     = 10 #kOhms

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in Vrl]
projection['ro'] = sensor2['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('','Volt', 'Ratio','Alcohol', 'Benzine', 'CH4', 'Hexane', 'LPG', 'CO'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])


    projection['benzine'] = {}
    projection['benzine']['p1'], projection['benzine']['p2'] = getPoints(r, _benzine)
    projection['benzine']['m'] = M(projection['benzine']['p1'], projection['benzine']['p2'])
    projection['benzine']['b'] = B(projection['benzine']['p1'], projection['benzine']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['hexane'] = {}
    projection['hexane']['p1'], projection['hexane']['p2'] = getPoints(r, _hexane)
    projection['hexane']['m'] = M(projection['hexane']['p1'], projection['hexane']['p2'])
    projection['hexane']['b'] = B(projection['hexane']['p1'], projection['hexane']['m'])


    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    if r > 9:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(Vrl[idx], 3),
                   round(r, 2),
                   round(MgL(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                   round(MgL(r, projection['benzine']['m'], projection['benzine']['b']),2),
                   round(MgL(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(MgL(r, projection['hexane']['m'], projection['hexane']['b']),2),
                   round(MgL(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(MgL(r, projection['co']['m'], projection['co']['b']),2),
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(Vrl[idx], 3),
                   round(r, 2),
                   round(MgL(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                   round(MgL(r, projection['benzine']['m'], projection['benzine']['b']),2),
                   "----",
                   round(MgL(r, projection['hexane']['m'], projection['hexane']['b']),2),
                   round(MgL(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(MgL(r, projection['co']['m'], projection['co']['b']),2),
                  ))    
    
    

    Volt     Ratio    Alcohol       Benzine       CH4           Hexane        LPG           CO           
0   0.48     60.0     0.0           0.0           0.0           0.06          0.05          0.04         
1   0.58     48.56    0.0           0.0           0.22          0.12          0.13          0.15         
2   0.68     40.48    0.0           0.0           6.15          0.2           0.28          0.48         
3   0.78     34.47    0.0           0.0           50.77         0.31          0.52          0.97         
4   0.88     29.83    0.0           0.0           267.07        0.46          1.04          1.62         
5   0.98     26.14    0.0           0.0           1218.35       0.68          1.53          2.55         
6   1.08     23.13    0.0           0.0           4963.87       0.97          2.17          3.89         
7   1.18     20.63    0.0           0.0           18458.01      1.37          3.03          5.78         
8   1.28     18.52    0.0           0.0       

In [12]:
## projectionRo - Fixing VRL and Varying the Ro

# Sensor 1

Vcc    = 5 #volts
Vrl    = sensor1['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor1['rs']   
projectionRo['ro'] = np.arange(sensor1['ro'] - sensor1['ro']/10 , sensor1['ro'] + sensor1['ro']/10 , 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format('', 'Volt','Rs', 'Ro', 'Ratio','Alcohol', 'Benzine', 'CH4', 'Hexane', 'LPG', 'CO'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])


    projection['benzine'] = {}
    projection['benzine']['p1'], projection['benzine']['p2'] = getPoints(r, _benzine)
    projection['benzine']['m'] = M(projection['benzine']['p1'], projection['benzine']['p2'])
    projection['benzine']['b'] = B(projection['benzine']['p1'], projection['benzine']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['hexane'] = {}
    projection['hexane']['p1'], projection['hexane']['p2'] = getPoints(r, _hexane)
    projection['hexane']['m'] = M(projection['hexane']['p1'], projection['hexane']['p2'])
    projection['hexane']['b'] = B(projection['hexane']['p1'], projection['hexane']['m'])


    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
   .format(idx,
           round(Vrl, 3),
           round(projectionRo['rs'],2),
           round(projectionRo['ro'][idx],2),
           round(r, 2),
           round(MgL(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
           round(MgL(r, projection['benzine']['m'], projection['benzine']['b']),2),
           round(MgL(r, projection['ch4']['m'], projection['ch4']['b']),2),
           round(MgL(r, projection['hexane']['m'], projection['hexane']['b']),2),
           round(MgL(r, projection['lpg']['m'], projection['lpg']['b']),2),
           round(MgL(r, projection['co']['m'], projection['co']['b']),2),
          ))
    

    Volt     Rs       Ro       Ratio    Alcohol  Benzine  CH4      Hexane   LPG      CO      
0   0.5      90.0     1.35     66.67    0.0      0.0      0.0      0.05     0.03     0.02    
1   0.5      90.0     1.45     62.07    0.0      0.0      0.0      0.06     0.04     0.03    
2   0.5      90.0     1.55     58.06    0.0      0.0      0.0      0.07     0.06     0.05    


In [13]:
## projectionRo - Fixing VRL and Varying the Ro

# Sensor 2

Vcc    = 5 #volts
Vrl    = sensor2['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor2['rs']   
projectionRo['ro'] = np.arange(sensor2['ro'] - sensor2['ro']/10 , sensor2['ro'] + sensor2['ro']/10 , 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format('', 'Volt','Rs', 'Ro', 'Ratio','Alcohol', 'Benzine', 'CH4', 'Hexane', 'LPG', 'CO'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])


    projection['benzine'] = {}
    projection['benzine']['p1'], projection['benzine']['p2'] = getPoints(r, _benzine)
    projection['benzine']['m'] = M(projection['benzine']['p1'], projection['benzine']['p2'])
    projection['benzine']['b'] = B(projection['benzine']['p1'], projection['benzine']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['hexane'] = {}
    projection['hexane']['p1'], projection['hexane']['p2'] = getPoints(r, _hexane)
    projection['hexane']['m'] = M(projection['hexane']['p1'], projection['hexane']['p2'])
    projection['hexane']['b'] = B(projection['hexane']['p1'], projection['hexane']['m'])


    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
   .format(idx,
           round(Vrl, 3),
           round(projectionRo['rs'],2),
           round(projectionRo['ro'][idx],2),
           round(r, 2),
           round(MgL(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
           round(MgL(r, projection['benzine']['m'], projection['benzine']['b']),2),
           round(MgL(r, projection['ch4']['m'], projection['ch4']['b']),2),
           round(MgL(r, projection['hexane']['m'], projection['hexane']['b']),2),
           round(MgL(r, projection['lpg']['m'], projection['lpg']['b']),2),
           round(MgL(r, projection['co']['m'], projection['co']['b']),2),
          ))
    

    Volt     Rs       Ro       Ratio    Alcohol  Benzine  CH4      Hexane   LPG      CO      
0   0.48     94.17    1.41     66.67    0.0      0.0      0.0      0.05     0.03     0.02    
1   0.48     94.17    1.51     62.26    0.0      0.0      0.0      0.06     0.04     0.03    
2   0.48     94.17    1.61     58.4     0.0      0.0      0.0      0.07     0.05     0.05    
3   0.48     94.17    1.71     54.99    0.0      0.0      0.0      0.08     0.07     0.07    


### Ro values to be considered

- __Sensor 1__: 1.500 kOhms
- __Sensor 2__: 1.569 kOhms 